In [81]:
import pandas as pd
import numpy as np
import pandas as pd
import itertools 
import matplotlib.pyplot as plt
from terminaltables import AsciiTable
from scipy.stats import chi2

In [82]:
#SE IMPORTA EL CSV
nio_data = pd.read_csv('../CSV/NIO2.csv')
nio_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-04-01,2.63,2.76,2.60,2.65,2.65,17153800
1,2020-04-02,2.51,2.59,2.22,2.39,2.39,58942700
2,2020-04-03,2.45,2.49,2.33,2.40,2.40,19519500
3,2020-04-06,2.48,2.55,2.40,2.47,2.47,34866700
4,2020-04-07,2.81,2.81,2.65,2.70,2.70,46149600
...,...,...,...,...,...,...,...
980,2024-02-23,5.67,5.68,5.37,5.40,5.40,75135200
981,2024-02-26,5.54,5.77,5.50,5.65,5.65,60295300
982,2024-02-27,5.81,5.93,5.72,5.78,5.78,53722100
983,2024-02-28,5.60,5.61,5.39,5.43,5.43,58390500


In [83]:
#LIMPIEZA DATOS
#SE BORRAN LAS COLUMNAS QUE NO SE USARAN
nio_data = nio_data.drop(columns=['Open', 'High', 'Low', 'Close','Volume'])
nio_data

,Date,Adj Close
0,2020-04-01,2.65
1,2020-04-02,2.39
2,2020-04-03,2.40
3,2020-04-06,2.47
4,2020-04-07,2.70
...,...,...
980,2024-02-23,5.40
981,2024-02-26,5.65
982,2024-02-27,5.78
983,2024-02-28,5.43


In [84]:
numero_periodos = 4
#SE CALCULA EL CAMBIO PORCENTUAL DE CADA REGISTRO DE ACUERDO AL REGISTRO ANTERIOR
percentage = [0,]
for i in range(1, len(nio_data)):
    percent = (nio_data['Adj Close'][i] - nio_data['Adj Close'][i-1])/nio_data['Adj Close'][i-1]
    percentage.append(percent)
nio_data.insert(2, 'Change %', percentage)

#SE CALCULA LA DESVIACION ESTANDAR DE LOS 20 DATOS ANTERIORES
std_dev = [0,]*20
for i in range(21, len(percentage) + 1):
    standar_dev = np.std(percentage[i-20:i])
    std_dev.append(standar_dev)
nio_data.insert(3, 'std dev', std_dev)

#SE ASIGNA EL ESTADO DE ACUERDO A LA DESVIACION ESTANDAR DE LAS 20 SEMANAS ANTERIORES
states = [0,]*20
for j in range(20, len(percentage)):
    change = percentage[j]
    desv = std_dev[j]
    if change <= -2*desv:
        state = "B3"
    elif -2*desv < change <= -desv:
        state = "B2"
    elif -desv < change < 0:
        state = "B1"
    elif 0 <= change < desv:
        state = "S1"
    elif desv <= change < 2*desv:
        state = "S2"
    elif 2*desv <= change:  
        state = "S3"
    states.append(state)
nio_data.insert(4, 'Current State', states)

#SE ASIGNA UNA NUEVA COLUMNA CON EL ESTADO SIGUIENTE AL REGISTRO
next_state_column=[]
for i in range(len(nio_data)-1):
    next_state_column.append(nio_data['Current State'][i+1])

previows_state_column=[0]
for i in range(1,len(nio_data)-1):
    previows_state_column.append(nio_data['Current State'][i-1])
    
#SE BORRA LA ULTIMA FILA POR QUE NO TIENE ESTADO SIGUIENTE
nio_data = nio_data.drop(nio_data.index[-1])
nio_data.insert(5, 'Next State', next_state_column)
nio_data.insert(4, 'Previous State', previows_state_column)

# complete_table = AsciiTable([ford_data.columns]+list(ford_data.values))
# print("Complete Table \n",complete_table.table)

ford_trim=nio_data.iloc[21:]
table_week_21 = AsciiTable([ford_trim.columns]+list(ford_trim.values))



ford_trim = ford_trim.copy()

# Calculate the number of rows per period
rows_per_period = len(ford_trim) // numero_periodos

# Create a new column to store the period
ford_trim['Period'] = 0

# Assign the period to each row based on the index
for i in range(5):
    start_index = i * rows_per_period
    end_index = (i + 1) * rows_per_period
    ford_trim.loc[start_index:end_index, 'Period'] = i + 1

# Correct the last period to be 5 instead of 0
ford_trim.loc[end_index:, 'Period'] = 5

# Access the AsciiTable object using an array divided by the period
table_by_period = []
for period in range(1, numero_periodos + 1):
    period_data = ford_trim[ford_trim['Period'] == period]
    table = AsciiTable([period_data.columns] + period_data.values.tolist())
    table_by_period.append(table)


for period, table in enumerate(table_by_period, start=1):
    print(f"Table for Period {period}\n{table.table}")





Table for Period 1
+------------+-----------+------------------------+----------------------+----------------+---------------+------------+--------+
| Date       | Adj Close | Change %               | std dev              | Previous State | Current State | Next State | Period |
+------------+-----------+------------------------+----------------------+----------------+---------------+------------+--------+
| 2020-05-01 | 3.18      | -0.06744868035190615   | 0.05159741300704764  | B2             | B2            | S1         | 1      |
| 2020-05-04 | 3.34      | 0.05031446540880494    | 0.05206088664125541  | B2             | S1            | B1         | 1      |
| 2020-05-05 | 3.28      | -0.017964071856287442  | 0.052565786063630486 | S1             | B1            | S2         | 1      |
| 2020-05-06 | 3.62      | 0.10365853658536595    | 0.053386504111405775 | B1             | S2            | S1         | 1      |
| 2020-05-07 | 3.63      | 0.0027624309392264602  | 0.05346623321768939

In [85]:
#FUNCIONES PARA SOLUCIONAR DIVISIONES POR 0
def resizeExpectedValues(freq_matrix, states):
    index_to_delete = []
    for index in range(len(freq_matrix)):
        if 0 in freq_matrix[index]:
            if index > 0:
                freq_matrix[index-1] += freq_matrix[index]
                freq_matrix[:,index-1] += freq_matrix[:,index] 
            else:
                freq_matrix[index+1] += freq_matrix[index]
                freq_matrix[:,index+1] += freq_matrix[:,index] 
            index_to_delete.append(index)
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=0)
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=1)
    
    for i in range(len(states)):
        if i in index_to_delete:
            if index > 0:
                states[i-1]=f"{states[i-1]}+{states[i]}"
            else:
                states[i+1]=f"{states[i]}+{states[i+1]}"
    states=np.delete(states,index_to_delete)
    return freq_matrix, states

In [86]:
frecuencias_observadas = []

table_by_period = np.array_split(ford_trim, numero_periodos)

for i in range(numero_periodos):
    # Calcula la tabla de frecuencias observadas usando crosstab entre el estado actual y el estado anterior (shift)
    frecuencia_obs = pd.crosstab(table_by_period[i]['Current State'].shift(), table_by_period[i]['Current State'], normalize=False)
    print("FRECUENCIA OBSERVADA - PERIODO", i+1, "\n", frecuencia_obs, "\n")
    # Agrega la frecuencia observada a la lista
    frecuencias_observadas.append(frecuencia_obs)


FRECUENCIA OBSERVADA - PERIODO 1 
 Current State  B1  B2  B3  S1  S2  S3
Current State                        
B1             33   9   0  32  13   3
B2              4   2   0  12   3   1
B3              3   0   0   0   0   0
S1             37   8   1  23  10   3
S2             10   2   2  12   4   2
S3              4   0   0   3   2   2 

FRECUENCIA OBSERVADA - PERIODO 2 
 Current State  B1  B2  B3  S1  S2  S3
Current State                        
B1             31  13   3  24  11   5
B2             10   7   2  11   2   1
B3              2   1   0   3   2   0
S1             29   8   2  22  11   3
S2             12   4   0  10   2   0
S3              3   0   1   5   0   0 

FRECUENCIA OBSERVADA - PERIODO 3 
 Current State  B1  B2  B3  S1  S2  S3
Current State                        
B1             39  14   1  26   6   1
B2              9   7   0  17   2   1
B3              1   0   0   1   3   0
S1             28  12   4  27   5   5
S2              6   3   0   7   5   1
S3              5

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [87]:
porcentaje_observado = [None] * numero_periodos

for i in range(0, numero_periodos):
    # Calcula los porcentajes observados dividiendo las frecuencias observadas por la suma total de cada periodo
    porcentaje_observado[i] = frecuencias_observadas[i] / frecuencias_observadas[i].sum()
    #Transpone la matriz para que sea más fácil de leer y comparar con el Excel
    porcentaje_observado[i] = porcentaje_observado[i].T
    
print("Porcentaje relativo:\n", porcentaje_observado[2]) #Ejemplo de impresion de un periodo


Porcentaje relativo:
 Current State        B1        B2        B3        S1        S2        S3
Current State                                                            
B1             0.443182  0.102273  0.011364  0.318182  0.068182  0.056818
B2             0.388889  0.194444  0.000000  0.333333  0.083333  0.000000
B3             0.200000  0.000000  0.000000  0.800000  0.000000  0.000000
S1             0.325000  0.212500  0.012500  0.337500  0.087500  0.025000
S2             0.272727  0.090909  0.136364  0.227273  0.227273  0.045455
S3             0.111111  0.111111  0.000000  0.555556  0.111111  0.111111


In [88]:
frecuencias_observadas = []
porcentaje_observado = [None] * numero_periodos

#Se calcula la tabla de frecuencias observadas para cada periodo
for i in range(numero_periodos):
    frecuencia_obs = pd.crosstab(table_by_period[i]['Current State'].shift(), table_by_period[i]['Current State'], normalize=False)
    frecuencias_obs = np.array(frecuencia_obs)    
    frecuencias_observadas.append(frecuencia_obs)


def calculateChiArgs(data):
    frecuencias_observadasPeriodo = []

    #PROBABILIDAD ESPERADA
    states=np.unique(data['Current State'])
    states_mapping = {}
    for i,state in enumerate(states):
        states_mapping[state]=i

    #SE CREA LA MATRIZ DE FRECUENCIAS ESPERADAS
    freq_expected=np.zeros(((len(states_mapping)),len(states_mapping)))
    for i in range(len(data)):
        freq_expected[states_mapping[data.iloc[i]['Current State']], states_mapping[data.iloc[i]['Next State']]] += 1
    
    freq_obs = np.zeros(((len(states_mapping)),len(states_mapping)))
    index_to_delete = []
    #FUNCION PARA FRECUENCIAS DE 0
    if 0 in freq_expected:
        freq_expected, states, index_to_delete =resizeExpectedValues(freq_expected,states)
        states_mapping={new_state: i for i,new_state in enumerate(states)}
    

    # rows = ['B1', 'B2+B3', 'S1', 'S2', 'S3']
    for i in range(0, numero_periodos):
        #Calculo de Frecuencias Observadas por periodo
        frecuencias_observadas[i] = resizeExpectedValuesObs(frecuencias_observadas[i].values, index_to_delete)
        #Calculo de porcentaje observado
        porcentaje_observado[i] = frecuencias_observadas[i] / frecuencias_observadas[i].sum()
        # porcentaje_observado[i] = pd.DataFrame(porcentaje_observado[i], columns=rows, index=rows)
        print(f"Porcentaje observado:\n{porcentaje_observado[i]}\n") #Ejemplo de impresion de un periodo

    #SE DETERMINA LA MATRIZ DE PROBABILIDAD ESPERADA
    transition_expected = np.zeros(((len(states)), len(states)))
    for i in range(len(freq_expected)):
        transition_expected[i] = freq_expected[i] / sum(freq_expected[i])

    data_frame_expected = pd.DataFrame(transition_expected, columns=states, index=states)
    print(f"Probabilidad Esperada\n{data_frame_expected}\n")
    return transition_expected
    

#FUNCIONES PARA SOLUCIONAR DIVISIONES POR 0
def resizeExpectedValues(freq_matrix, states):

    index_to_delete = []
    for index in range(len(freq_matrix)):
        if 0 in freq_matrix[index]:
            if index > 0:
                freq_matrix[index-1] += freq_matrix[index]
                freq_matrix[:,index-1] += freq_matrix[:,index] 
            else:
                freq_matrix[index+1] += freq_matrix[index]
                freq_matrix[:,index+1] += freq_matrix[:,index] 
            index_to_delete.append(index)
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=0)
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=1)
    
    for i in range(len(states)):
        if i in index_to_delete:
            if i > 0:
                states[i-1]=f"{states[i-1]}+{states[i]}"
            else:
                states[i+1]=f"{states[i]}+{states[i+1]}"
    states=np.delete(states,index_to_delete)
    return freq_matrix, states, index_to_delete

#Cambia el tamaño de la matriz de frecuencias observadas para mantener disposición de columnas y filas reducidas en frecuencias esperadas
def resizeExpectedValuesObs(freq_matrix, index_to_delete):
    for i in range(len(freq_matrix)):
        if i in index_to_delete:
            if i > 0:
                
                freq_matrix[i-1]=freq_matrix[i-1]+freq_matrix[i]
                freq_matrix[:,i-1] = freq_matrix[:,i-1] + freq_matrix[:,i]
            else:
                freq_matrix[i+1]=freq_matrix[i]+freq_matrix[i+1]
                freq_matrix[:,i+1] = freq_matrix[:,i+1] + freq_matrix[:,i]
        
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=0)
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=1)
    
    return freq_matrix
probabilidad_observada = []
probabilidad_observada = calculateChiArgs(ford_trim[['Previous State', 'Current State', 'Next State']])

Porcentaje observado:
[[0.1375     0.0375     0.13333333 0.05416667 0.0125    ]
 [0.02916667 0.00833333 0.05       0.0125     0.00416667]
 [0.15416667 0.0375     0.09583333 0.04166667 0.0125    ]
 [0.04166667 0.01666667 0.05       0.01666667 0.00833333]
 [0.01666667 0.         0.0125     0.00833333 0.00833333]]

Porcentaje observado:
[[0.12916667 0.06666667 0.1        0.04583333 0.02083333]
 [0.05       0.04166667 0.05833333 0.01666667 0.00416667]
 [0.12083333 0.04166667 0.09166667 0.04583333 0.0125    ]
 [0.05       0.01666667 0.04166667 0.00833333 0.        ]
 [0.0125     0.00416667 0.02083333 0.         0.        ]]

Porcentaje observado:
[[0.1625     0.0625     0.10833333 0.025      0.00416667]
 [0.04166667 0.02916667 0.075      0.02083333 0.00416667]
 [0.11666667 0.06666667 0.1125     0.02083333 0.02083333]
 [0.025      0.0125     0.02916667 0.02083333 0.00416667]
 [0.02083333 0.         0.00833333 0.00416667 0.00416667]]

Porcentaje observado:
[[0.13389121 0.05020921 0.13389121 0

In [89]:
ChiObs = 0
for i in range(0, numero_periodos): 
     # Suma acumulativa de los valores chi-cuadrado observados se aplica la formula vista en clase
    ChiObs+= np.sum(frecuencias_observadas[i].sum()*((porcentaje_observado[i]-probabilidad_observada)**2/probabilidad_observada))
    break

print(f'Chi Observado: {ChiObs}')
sqrt = np.sqrt(ChiObs)


Chi Observado: 791.3672929543785


In [90]:
from scipy.stats import chi2
grados_libertad_total = 0
sum_filas = 0
# Iteración a través de las frecuencias observadas
for df in frecuencias_observadas:
    num_filas, num_columnas = df.shape
    # Suma el número de filas al total para los grados de libertad
    sum_filas += num_filas
# Calcula los grados de libertad
grados_libertad = (sum_filas - 1) * (num_columnas - 1)

print("Grados de libertad total:", grados_libertad)
# Calcula el valor chi-cuadrado teórico para un nivel de significancia del 5% y los grados de libertad calculados
chiTeorico = chi2.ppf(0.95, grados_libertad)
print("Chi Teorico= ",chiTeorico)

Grados de libertad total: 76
Chi Teorico=  97.35097037903296


In [91]:
# Compara el valor chi-cuadrado observado con el valor chi-cuadrado teórico crítico
if(ChiObs < chiTeorico):
    print("Se acepta la hipótesis nula, por lo tanto se considera estacionario en el tiempo.")
else:
    print("Se rechaza la hipótesis nula, por lo tanto no se considera estacionario en el tiempo.")

Se rechaza la hipótesis nula, por lo tanto no se considera estacionario en el tiempo.
